# 05 - Adaptive Weight Scheduling Simulation

Objective: Determine optimal γ_base and η for curriculum learning.

## Formula
γ_t = min(γ_max, γ_base + η × t)

## Recommended Configuration (from Algorithm 1)
- γ_base = 0.3 (30% geometric, 70% semantic initially)
- η = 0.0001 (reaches γ_t=0.8 at step ~5000)
- γ_max = 0.8 (80% geometric, 20% semantic at convergence)

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
from src.rewards import CompositeReward

device = torch.device('cpu')  # CPU is fine for this simulation

## 1. Visualize γ_t Schedules

In [ ]:
def compute_gamma_schedule(gamma_base, eta, gamma_max, max_steps=10000):
    """Compute γ_t for all training steps."""
    steps = np.arange(0, max_steps + 1, 100)
    gamma_t = np.minimum(gamma_max, gamma_base + eta * steps)
    return steps, gamma_t


# Test different configurations
configs = [
    {'gamma_base': 0.2, 'eta': 0.00005, 'gamma_max': 0.8, 'label': 'Conservative (slow)'},
    {'gamma_base': 0.3, 'eta': 0.0001, 'gamma_max': 0.8, 'label': 'Recommended'},
    {'gamma_base': 0.4, 'eta': 0.0001, 'gamma_max': 0.8, 'label': 'Higher initial γ'},
    {'gamma_base': 0.3, 'eta': 0.0002, 'gamma_max': 0.8, 'label': 'Faster increase'},
    {'gamma_base': 0.5, 'eta': 0.0001, 'gamma_max': 0.9, 'label': 'Aggressive'},
]

In [ ]:
plt.figure(figsize=(12, 6))

for config in configs:
    steps, gamma_t = compute_gamma_schedule(
        config['gamma_base'],
        config['eta'],
        config['gamma_max']
    )
    
    linestyle = '-' if config['label'] == 'Recommended' else '--'
    linewidth = 2.5 if config['label'] == 'Recommended' else 1.5
    
    plt.plot(steps, gamma_t, label=config['label'], 
             linestyle=linestyle, linewidth=linewidth)

plt.xlabel('Training Step', fontsize=12)
plt.ylabel('γ_t (Geometric Weight)', fontsize=12)
plt.title('Adaptive Weight Scheduling: γ_t over Training', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)

# Add annotations
plt.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)
plt.text(100, 0.52, 'Equal weighting', fontsize=9, alpha=0.7)

plt.tight_layout()
plt.savefig('../results/gamma_schedules.png', dpi=150, bbox_inches='tight')
plt.show()

## 2. Semantic vs Geometric Weight Over Time

In [ ]:
# Use recommended config
gamma_base, eta, gamma_max = 0.3, 0.0001, 0.8
steps, gamma_t = compute_gamma_schedule(gamma_base, eta, gamma_max)
semantic_weight = 1 - gamma_t

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Stacked area
axes[0].fill_between(steps, 0, semantic_weight, alpha=0.7, label='Semantic (R_sem)', color='blue')
axes[0].fill_between(steps, semantic_weight, 1, alpha=0.7, label='Geometric (R_geom)', color='orange')
axes[0].set_xlabel('Training Step')
axes[0].set_ylabel('Relative Weight')
axes[0].set_title('Weight Distribution Over Training')
axes[0].legend(loc='right')
axes[0].set_ylim(0, 1)
axes[0].grid(True, alpha=0.3)

# Right: Lines
axes[1].plot(steps, gamma_t, label='γ_t (Geometric)', linewidth=2, color='orange')
axes[1].plot(steps, semantic_weight, label='1-γ_t (Semantic)', linewidth=2, color='blue')
axes[1].axvline(x=5000, color='red', linestyle='--', alpha=0.7, label='γ_t reaches max')
axes[1].set_xlabel('Training Step')
axes[1].set_ylabel('Weight')
axes[1].set_title('Weight Curves')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/weight_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Key Training Phases

In [ ]:
# Compute γ at key steps
key_steps = [0, 1000, 2000, 3000, 5000, 7500, 10000]

print("Training Phase Analysis (Recommended: γ_base=0.3, η=0.0001)")
print("="*60)
print(f"{'Step':>8} | {'γ_t':>8} | {'Semantic':>8} | {'Phase':>20}")
print("-"*60)

for step in key_steps:
    gt = min(gamma_max, gamma_base + eta * step)
    sem = 1 - gt
    
    if step < 1000:
        phase = "Early: Focus semantic"
    elif step < 3000:
        phase = "Transition: Balanced"
    elif step < 5000:
        phase = "Mid: Increasing geom"
    else:
        phase = "Late: Focus geometric"
    
    print(f"{step:>8} | {gt:>8.3f} | {sem:>8.3f} | {phase:>20}")

## 4. Effect on Reward Computation

In [ ]:
# Simulate how different γ_t affects final reward
# Assume hypothetical r_geom and r_sem values

r_geom_values = [0.6, 0.7, 0.8, 0.9]
r_sem_values = [0.7, 0.8, 0.85, 0.9]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, (r_geom, r_sem) in enumerate(zip(r_geom_values, r_sem_values)):
    ax = axes[idx // 2, idx % 2]
    
    steps_plot = np.linspace(0, 10000, 100)
    gamma_t_plot = np.minimum(gamma_max, gamma_base + eta * steps_plot)
    
    # R_total = r_geom^γ_t × r_sem^(1-γ_t)
    r_total = (r_geom ** gamma_t_plot) * (r_sem ** (1 - gamma_t_plot))
    
    ax.plot(steps_plot, r_total, 'b-', linewidth=2)
    ax.axhline(y=r_geom, color='orange', linestyle='--', alpha=0.7, label=f'R_geom = {r_geom}')
    ax.axhline(y=r_sem, color='green', linestyle='--', alpha=0.7, label=f'R_sem = {r_sem}')
    
    ax.set_xlabel('Training Step')
    ax.set_ylabel('R_total')
    ax.set_title(f'Reward Evolution (R_geom={r_geom}, R_sem={r_sem})')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0.5, 1.0)

plt.suptitle('Effect of Curriculum on Total Reward', fontsize=14)
plt.tight_layout()
plt.savefig('../results/reward_evolution.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Configuration Comparison Table

In [ ]:
import pandas as pd

comparison_data = []

for config in configs:
    gb, et, gm = config['gamma_base'], config['eta'], config['gamma_max']
    
    # Step at which γ_t reaches max
    steps_to_max = (gm - gb) / et if et > 0 else float('inf')
    
    # γ_t at key points
    g_1000 = min(gm, gb + et * 1000)
    g_5000 = min(gm, gb + et * 5000)
    g_10000 = min(gm, gb + et * 10000)
    
    comparison_data.append({
        'Config': config['label'],
        'γ_base': gb,
        'η': et,
        'γ_max': gm,
        'Steps to Max': int(steps_to_max),
        'γ @ 1K': f'{g_1000:.2f}',
        'γ @ 5K': f'{g_5000:.2f}',
        'γ @ 10K': f'{g_10000:.2f}'
    })

config_df = pd.DataFrame(comparison_data)
display(config_df)

## 6. Recommendations

Based on the analysis:

### **Recommended Configuration**
- **γ_base = 0.3**: Start with 30% geometric weight to allow semantic consistency to establish first
- **η = 0.0001**: Gradually increase geometric weight
- **γ_max = 0.8**: Cap at 80% geometric to maintain some semantic guidance

### **Rationale**
1. **Early training (0-1000 steps)**: Higher semantic weight prevents Janus problem
2. **Mid training (1000-5000 steps)**: Gradual shift balances both objectives
3. **Late training (5000+ steps)**: Geometric dominance enforces 3D consistency

### **Alternative Configurations**
- **Conservative**: Use γ_base=0.2, η=0.00005 if Janus problem is severe
- **Aggressive**: Use γ_base=0.4, η=0.0002 for faster convergence on well-behaved data

In [ ]:
# Save recommendations
recommendations = {
    'recommended': {
        'gamma_base': 0.3,
        'eta': 0.0001,
        'gamma_max': 0.8,
        'description': 'Balanced curriculum for most cases'
    },
    'conservative': {
        'gamma_base': 0.2,
        'eta': 0.00005,
        'gamma_max': 0.8,
        'description': 'For severe Janus problem cases'
    },
    'aggressive': {
        'gamma_base': 0.4,
        'eta': 0.0002,
        'gamma_max': 0.9,
        'description': 'For faster convergence on clean data'
    }
}

import json
with open('../results/weight_scheduling_recommendations.json', 'w') as f:
    json.dump(recommendations, f, indent=2)

print("Saved recommendations to ../results/weight_scheduling_recommendations.json")